<a href="https://colab.research.google.com/github/rodrigosarloaf/projeto_DL_23_1/blob/main/junta_prodes_deter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Pacotes
import pathlib
from osgeo import ogr, gdal, gdalconst
from skimage.morphology import disk, dilation, erosion
import os
import numpy as np
import shutil
from google.colab import drive
import cv2
from datetime import date, timedelta
import pandas as pd
import pickle

In [ ]:
# Conectando ao drive
drive.mount('/content/drive', force_remount=True)
root_path = '/content/drive/My Drive/projeto_DL/'
os.chdir(root_path)

Mounted at /content/drive


In [ ]:
# Juntando as séries de PRODES e DETER para serem usadas nos modelos
# Veja que o PRODES termina em 30/07 - é a data de referência para taxas de desmatamento

# Tratamendo de dados semanais
# Puxando csv de referência
df_semanal = pd.read_csv(
    '/content/drive/My Drive/projeto_DL/deter_tratado/referencia_semana.csv'
    ).drop(columns = 'Unnamed: 0')
df_semanal['begin'] = pd.to_datetime(df_semanal['begin'])
df_semanal['end'] = pd.to_datetime(df_semanal['end'])
df_semanal.head()

,begin,end,name,name_2
0,2016-08-07,2016-08-13,83,0
1,2016-08-14,2016-08-20,84,1
2,2016-08-21,2016-08-27,85,2
3,2016-08-28,2016-09-03,86,3
4,2016-09-04,2016-09-10,87,4


In [ ]:
# Jogando fora as primeiras linhas que são do "ano-PRODES" anterior ao começo da nossa análise
# Na verdade já sabemos que a amostra deve começar após o "ano-PRODES" que se inicia no meio de 2016
df_semanal = df_semanal.loc[df_semanal.begin >= pd.Timestamp(2016,8,1)]
# Jogando fora as últimas linhas que são do "ano-PRODES" posterior ao fim da nossa análise
df_semanal = df_semanal.loc[df_semanal.begin < pd.Timestamp(2021,8,1)]
#
df_semanal = df_semanal.reset_index(drop=True)
df_semanal = df_semanal.sort_values(by = ['begin'])
# Atualizando os nomes e salvando
df_semanal['name_2'] = range(len(df_semanal)) # np.arange(1, len(df_semanal), 1)
df_semanal.to_csv('/content/drive/My Drive/projeto_DL/deter_tratado/referencia_semana.csv')
#
df_semanal.head()

,begin,end,name,name_2
0,2016-08-07,2016-08-13,83,0
1,2016-08-14,2016-08-20,84,1
2,2016-08-21,2016-08-27,85,2
3,2016-08-28,2016-09-03,86,3
4,2016-09-04,2016-09-10,87,4


In [ ]:
path_semana = '/content/drive/My Drive/projeto_DL/deter_tratado/semanal/'
path_prodes = "/content/drive/My Drive/projeto_DL/prodes_tratado/"

anos = [2016,2017,2018,2019,2020,2021]
for i in anos:
  # Filtrando df interno para operarmos nele
  df_interno = df_semanal.loc[df_semanal.begin >= pd.Timestamp(i,8,1)].copy()
  df_interno = df_interno.loc[df_interno.begin < pd.Timestamp((i+1),8,1)]
  df_interno = df_interno.reset_index(drop=True)
  # Lendo o PRODES adequado
  with open(path_prodes+'prodes_'+str(i)+'.pkl', 'rb') as handle:
    prodes_atual = pickle.load(handle)

  for j in range(len(df_interno)):
    # Abrindo o DETER
    imagem_deter = cv2.imread(path_semana + str(df_interno['name'][j]) + '.tif', cv2.IMREAD_GRAYSCALE)
    imagem_deter = imagem_deter.astype('float64')
    # Acrescentando os alertas ao PRODES
    prodes_atual = np.add(prodes_atual, imagem_deter)
    # Possíveis erros
    if(np.sum(prodes_atual[prodes_atual > 2])):
      prodes_atual[prodes_atual > 2] = 2

    # Salvando
    with open("/content/drive/My Drive/projeto_DL/dataset_tratado/semana/"+str(i)+"_"+str(j)+".pkl", 'wb') as handle:
      pickle.dump(prodes_atual, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # Transformando os alertas de hoje em alertas antigos/não floresta
    if(np.sum(prodes_atual[prodes_atual == 1])):
      prodes_atual[prodes_atual == 1] = 2

  print("ano = " + str(i))


ano = 2016
ano = 2017
ano = 2018
ano = 2019
ano = 2020
ano = 2021


In [ ]:
# Checando
path_checar = "/content/drive/My Drive/projeto_DL/dataset_tratado/semana/"
print(len(os.listdir(path_checar))==len(df_semanal))

True
